# 4강. 나만의 RNN을 만들어보자 (feat.훈민정음서문)

https://www.youtube.com/watch?v=cdGBloT9vDk&list=PLfGJDDf2OqlQkHqKB7uonQGeNRfUo_TMe&index=22

저자의 노트북

- https://github.com/phdshinai/ANN_DL101/blob/main/DeepLearning101/VanillaRNN.ipynb

In [1]:
import numpy as np
import re


In [2]:
data = """
나라의 말이 중국과 달라 문자와 서로 통하지 아니하기에 이런 까닭으로 어리석은 백성이 이르고자 할 바가 있어도 마침내 제 뜻을 능히 펴지 못할 사람이 많으니라 내가 이를 위해 가엾이 여겨 새로 스물여덟 글자를 만드노니 사람마다 하여 쉬이 익혀 날로 씀에 편안케 하고자 할 따름이니라
"""

In [3]:
# --- linter를 위해 먼저 선언함.

# 기본적인 parameters
epochs = 10000
h_size = 100
seq_len = 3
learning_rate = 1e-2

In [4]:
def data_preprocessing(data):
    data = re.sub('[^가-힣]', ' ', data)
    tokens = data.split()
    vocab = list(set(tokens))
    vocab_size = len(vocab)

    word_to_ix = {word: i for i, word in enumerate(vocab)}
    ix_to_word = {i: word for i, word in enumerate(vocab)}

    return tokens, vocab_size, word_to_ix, ix_to_word

In [5]:
# --- linter를 위해 먼저 실행

tokens, vocab_size, word_to_ix, ix_to_word = data_preprocessing(data)


In [6]:
def init_weights(h_size, vocab_size):
    U = np.random.randn(h_size, vocab_size) * 0.01
    W = np.random.randn(h_size, h_size) * 0.01
    V = np.random.randn(vocab_size, h_size) * 0.01
    return U,W,V

In [7]:
# --- liter

U, W, V = init_weights(h_size, vocab_size)

In [8]:
# 순방향
# 입력으로 '나라의' 가 들어오면, 타겟으로 '말이' 가 나오는 것을 기대하는 것임.
# hprev 는 코드 상으로는 이전 단계에서 backward()의 결과로 받는 것인데.. 정확히 무슨 의미??

def feedforward(inputs, targets, hprev):
    loss = 0
    xs, hs, ps, ys = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)

    # seq_len 는 몇 개의 단어를 한번에 묶어서 처리할 것인지 를 의미. ???
    # 만약 seq_len 이 3 이라면:
    #   input 으로는 ['나라의', '말이', '중국과'] 이렇게 3개가 들어오게 된다.
    #   그렇다면 target으로는 '달라' 가 나와야 할 것임..
    for i in range(seq_len):
        xs[i] = np.zeros((vocab_size, 1))
        xs[i][inputs[i]] = 1  # 각각의 word에 대한 one hot coding
        hs[i] = np.tanh(np.dot(U, xs[i]) + np.dot(W, hs[i - 1]))
        ys[i] = np.dot(V, hs[i])
        ps[i] = np.exp(ys[i]) / np.sum(np.exp(ys[i]))  # softmax계산
        loss += -np.log(ps[i][targets[i], 0])
    return loss, ps, hs, xs

In [9]:
def backward(ps, hs, xs):

    # Backward propagation through time (BPTT)
    # 처음에 모든 가중치들은 0으로 설정
    dV = np.zeros(V.shape)
    dW = np.zeros(W.shape)
    dU = np.zeros(U.shape)

    #-- 시간 순의 역순으로 계산을 해 나가야 한다.
    for i in range(seq_len)[::-1]:
        output = np.zeros((vocab_size, 1))
        output[targets[i]] = 1
            # output 의 one-hot encoding 이 ground-truth (정답)이다. 이걸 미리 만들어 둔다.

        # 참고: loss 값 L을 직접 구할 필요가 없다. 어차피 gradient를 구하는데 L값 자체가 필요하진 않다.

        # ps(확률). y_hat
        ps[i] = ps[i] - output.reshape(-1, 1)
        # 매번 i스텝에서 dL/dVi를 구하기
        dV_step_i = ps[i] @ (hs[i]).T  # (y_hat - y) @ hs.T - for each step

        dV = dV + dV_step_i  # dL/dVi를 다 더하기

        # 각i별로 V와 W를 구하기 위해서는
        # 먼저 공통적으로 계산되는 부분을 delta로 해서 계산해두고
        # 그리고 시간을 거슬러 dL/dWij와 dL/dUij를 구한 뒤
        # 각각을 합하여 dL/dW와 dL/dU를 구하고
        # 다시 공통적으로 계산되는 delta를 업데이트

        # i번째 스텝에서 공통적으로 사용될 delta
        delta_recent = (V.T @ ps[i]) * (1 - hs[i] ** 2)

        # 시간을 거슬러 올라가서 dL/dW와 dL/dU를 구하
        # for j in range(i + 1)[::-1]:
        for j in range(i, -1, -1): # i 부터 0 까지.
            dW_ij = delta_recent @ hs[j - 1].T

            dW = dW + dW_ij

            dU_ij = delta_recent @ xs[j].reshape(1, -1)
            dU = dU + dU_ij

            # 그리고 다음번 j번째 타임에서 공통적으로 계산할 delta를 업데이트
            delta_recent = (W.T @ delta_recent) * (1 - hs[j - 1] ** 2)

        for d in [dU, dW, dV]:
            np.clip(d, -1, 1, out=d)
    return dU, dW, dV, hs[len(inputs) - 1]

In [10]:
# word 는 입력. 예: '나라의'
# length 는 생성해 낼 출력 길이.
#
def predict(word, length):
    x = np.zeros((vocab_size, 1))
    x[word_to_ix[word]] = 1
    ixes = []
    h = np.zeros((h_size,1))

    for t in range(length):
        h = np.tanh(np.dot(U, x) + np.dot(W, h))
        y = np.dot(V, h)
        p = np.exp(y) / np.sum(np.exp(y))    # 소프트맥스
        ix = np.argmax(p)                    # 가장 높은 확률의 index를 리턴
        x = np.zeros((vocab_size, 1))        # 다음번 input x를 준비
        x[ix] = 1
        ixes.append(ix)
    pred_words = ' '.join(ix_to_word[i] for i in ixes)
    return pred_words

In [11]:
# 기본적인 parameters
epochs = 10000
h_size = 100
seq_len = 3
learning_rate = 1e-2

In [12]:
tokens, vocab_size, word_to_ix, ix_to_word = data_preprocessing(data)


In [13]:
tokens

['나라의',
 '말이',
 '중국과',
 '달라',
 '문자와',
 '서로',
 '통하지',
 '아니하기에',
 '이런',
 '까닭으로',
 '어리석은',
 '백성이',
 '이르고자',
 '할',
 '바가',
 '있어도',
 '마침내',
 '제',
 '뜻을',
 '능히',
 '펴지',
 '못할',
 '사람이',
 '많으니라',
 '내가',
 '이를',
 '위해',
 '가엾이',
 '여겨',
 '새로',
 '스물여덟',
 '글자를',
 '만드노니',
 '사람마다',
 '하여',
 '쉬이',
 '익혀',
 '날로',
 '씀에',
 '편안케',
 '하고자',
 '할',
 '따름이니라']

In [14]:
ix_to_word


{0: '가엾이',
 1: '까닭으로',
 2: '하여',
 3: '이를',
 4: '서로',
 5: '스물여덟',
 6: '글자를',
 7: '있어도',
 8: '능히',
 9: '중국과',
 10: '따름이니라',
 11: '하고자',
 12: '문자와',
 13: '이런',
 14: '달라',
 15: '여겨',
 16: '위해',
 17: '나라의',
 18: '사람마다',
 19: '많으니라',
 20: '날로',
 21: '내가',
 22: '통하지',
 23: '아니하기에',
 24: '못할',
 25: '이르고자',
 26: '마침내',
 27: '어리석은',
 28: '만드노니',
 29: '씀에',
 30: '바가',
 31: '익혀',
 32: '펴지',
 33: '할',
 34: '쉬이',
 35: '백성이',
 36: '제',
 37: '사람이',
 38: '뜻을',
 39: '편안케',
 40: '말이',
 41: '새로'}

In [15]:
U, W, V = init_weights(h_size, vocab_size)


In [16]:
p = 0
hprev = np.zeros((h_size, 1))
for epoch in range(epochs):

    for p in range(len(tokens)-seq_len):
        inputs = [word_to_ix[tok] for tok in tokens[p:p + seq_len]]
        targets = [word_to_ix[tok] for tok in tokens[p + 1:p + seq_len + 1]]

        loss, ps, hs, xs = feedforward(inputs, targets, hprev)

        dU, dW, dV, hprev = backward(ps, hs, xs)

        # Update weights and biases using gradient descent
        W -= learning_rate * dW
        U -= learning_rate * dU
        V -= learning_rate * dV

        # p += seq_len

    if epoch % 100 == 0:
        print(f'epoch {epoch}, loss: {loss}')

epoch 0, loss: 11.212394506604594
epoch 100, loss: 2.0089843922676285
epoch 200, loss: 0.2770074302725638
epoch 300, loss: 0.13434186810313714
epoch 400, loss: 0.08719279150149263
epoch 500, loss: 0.06239213672953292
epoch 600, loss: 0.04685421144339408
epoch 700, loss: 0.036932562015630825
epoch 800, loss: 0.030454957073053096
epoch 900, loss: 0.02603539851460957
epoch 1000, loss: 0.022822474792935293
epoch 1100, loss: 0.020315060298593562
epoch 1200, loss: 0.018242253821771415
epoch 1300, loss: 0.016471865882325437
epoch 1400, loss: 0.014944055365309263
epoch 1500, loss: 0.013624305990495383
epoch 1600, loss: 0.012482937925428008
epoch 1700, loss: 0.01149335588185963
epoch 1800, loss: 0.010633884220709499
epoch 1900, loss: 0.009887281573957319
epoch 2000, loss: 0.009239127277243422
epoch 2100, loss: 0.008676636529778228
epoch 2200, loss: 0.008188169609794781
epoch 2300, loss: 0.007763158383876019
epoch 2400, loss: 0.007392172006623667
epoch 2500, loss: 0.0070669668471854165
epoch 260

In [ ]:
for i in range(5):
    try:
        user_input = '나라의' # #input("input: ")
        if user_input == 'break':
            break
        response = predict(user_input,40)
        print(response)
    except:
        print('Uh oh try again!')